In [21]:
import time
# def keywordsMapper(elem):
    

def dcTransformMapper(elem):
    if elem == 'False':
        return [0]
    else:
        return [1]

from pyspark.sql import Row
from pyspark.ml.linalg import Vectors
def f(x):
    rel = {}
    rel['final_status'] = str(x[7])
    rel['features'] = Vectors.dense(float(x[0]),float(x[1]),float(x[2]),float(x[3]),float(x[4]),float(x[5]),float(x[6]))
    return rel

import csv
# remove first row(header)
trainKickstarter = sc.textFile("funding-successful-projects-on-kickstarter/train.csv")
trainHeader = trainKickstarter.first()
trainKickstarter = trainKickstarter.filter(lambda line: line != trainHeader).mapPartitions(lambda x: csv.reader(x))
# print(trainKickstarter.take(10))

# Data Transformations and filter lines with empty strings
trainKickstarter=trainKickstarter.map(lambda line: line[3:4]+dcTransformMapper(line[5])+line[8:14])
# print(trainKickstarter.take(10))
# trainKickstarter=trainKickstarter.filter(lambda line: line[3] != '' ).filter(lambda line: line[4] != '' )
trainKickstarter=trainKickstarter.map(lambda p: Row(**f(p))).toDF()

trainKickstarter.show(10)


+--------------------+------------+
|            features|final_status|
+--------------------+------------+
|[20.0,1.0,1.24133...|           1|
|[300.0,1.0,1.2424...|           0|
|[30.0,1.0,1.24302...|           0|
|[500.0,1.0,1.2435...|           1|
|[2000.0,1.0,1.243...|           0|
|[700.0,1.0,1.2438...|           0|
|[250.0,1.0,1.2438...|           0|
|[1000.0,1.0,1.244...|           1|
|[5000.0,1.0,1.244...|           0|
|[3500.0,1.0,1.244...|           0|
+--------------------+------------+
only showing top 10 rows



In [19]:
# RandomForest
from pyspark.ml import Pipeline
from pyspark.ml.feature import IndexToString,StringIndexer,VectorIndexer
from pyspark.ml.classification import RandomForestClassifier

startTime = time.time()

# Index labels, adding metadata to the label column.
# Fit on whole dataset to include all labels in index.
labelIndexer = StringIndexer(inputCol="final_status", outputCol="indexedLabel").fit(trainKickstarter)
# labelIndexer = StringIndexer().setInputCol("label").setOutputCol("indexedLabel").fit(trainTitanic)

# Automatically identify categorical features, and index them.
featureIndexer = VectorIndexer(inputCol="features", outputCol="indexedFeatures").fit(trainKickstarter)
# featureIndexer = VectorIndexer().setInputCol("features").setOutputCol("indexedFeatures").fit(trainTitanic)


# Split the data into training and test sets (30% held out for testing)
(trainingData, testData) = trainKickstarter.randomSplit([0.7, 0.3])

# Train a RandomForest model.
rf = RandomForestClassifier(labelCol="indexedLabel", featuresCol="indexedFeatures", numTrees=10)

# Convert indexed labels back to original labels.
labelConverter = IndexToString(inputCol="prediction", outputCol="predictedLabel",labels=labelIndexer.labels)


# Chain indexers and forest in a Pipeline
Pipeline =  Pipeline().setStages([labelIndexer, featureIndexer, rf, labelConverter])
# Train model.  This also runs the indexers.
PipelineModel = Pipeline.fit(trainingData)

endTime = time.time()
executeTime = endTime - startTime
print ('Execution Time For Random Forest = %s' % str(executeTime) + ' Seconds\n')

# Make predictions.
Predictions = PipelineModel.transform(testData)


preRel = Predictions.select("predictedLabel", "final_status", "features", "probability").collect()

print('final_status, [goal, deadline, state_change, created_at, launched_at]-->prob=[prob1,prob2], predicted_final_status')
for item in preRel:
    print(str(item['final_status'])+','+str(item['features'])+'-->prob='+str(item['probability'])+',predicted_final_status: '+str(item['predictedLabel']))
 


Execution Time For Random Forest = 13.67988109588623 Seconds

final_status, [goal, deadline, state_change, created_at, launched_at]-->prob=[prob1,prob2], predicted_final_status
1,[1.0,1.0,1313840898.0,1313840898.0,1307598064.0,1307792898.0,24.0]-->prob=[0.35661753991311823,0.6433824600868818],predicted_final_status: 1
0,[1.0,1.0,1313969001.0,1309917788.0,1308261188.0,1309735401.0,8.0]-->prob=[0.5257094637936314,0.4742905362063685],predicted_final_status: 0
0,[1.0,1.0,1314044880.0,1314044882.0,1308169690.0,1309280864.0,0.0]-->prob=[0.8069880273751915,0.19301197262480857],predicted_final_status: 0
1,[1.0,1.0,1322533897.0,1322533897.0,1319902947.0,1319938297.0,8.0]-->prob=[0.566105680826179,0.43389431917382104],predicted_final_status: 0
0,[1.0,1.0,1327780547.0,1325548636.0,1324486962.0,1325188547.0,0.0]-->prob=[0.8171919547876195,0.1828080452123806],predicted_final_status: 0
1,[1.0,1.0,1328826621.0,1328826626.0,1316658833.0,1324852221.0,25.0]-->prob=[0.3427625850023992,0.6572374149976008]

0,[652.0,1.0,1369051200.0,1369051238.0,1367194254.0,1367864724.0,8.0]-->prob=[0.8677246067668022,0.13227539323319784],predicted_final_status: 0
0,[660.0,1.0,1322011898.0,1322011898.0,1318478198.0,1318552298.0,8.0]-->prob=[0.8548812033383572,0.14511879666164293],predicted_final_status: 0
0,[660.0,1.0,1339781243.0,1339781243.0,1336064055.0,1336670843.0,0.0]-->prob=[0.8475466718021849,0.152453328197815],predicted_final_status: 0
1,[660.0,1.0,1368728246.0,1368728247.0,1365626706.0,1366136246.0,52.0]-->prob=[0.23404566621975725,0.7659543337802427],predicted_final_status: 1
0,[665.0,1.0,1293592859.0,1293592859.0,1289515036.0,1290568859.0,4.0]-->prob=[0.8885357915284977,0.11146420847150225],predicted_final_status: 0
0,[665.0,1.0,1340938992.0,1340938992.0,1336478218.0,1338346992.0,0.0]-->prob=[0.8475466718021849,0.152453328197815],predicted_final_status: 0
0,[665.0,1.0,1366921743.0,1366921744.0,1357233995.0,1364329743.0,2.0]-->prob=[0.9557304312070775,0.044269568792922535],predicted_final_stat

0,[1250.0,1.0,1296764580.0,1296764582.0,1289988872.0,1291069398.0,19.0]-->prob=[0.3691290619487402,0.6308709380512598],predicted_final_status: 1
0,[1250.0,1.0,1298235638.0,1298235639.0,1295640360.0,1295643638.0,20.0]-->prob=[0.3691290619487402,0.6308709380512598],predicted_final_status: 1
1,[1250.0,1.0,1298962740.0,1298962742.0,1295631321.0,1295797432.0,11.0]-->prob=[0.8622658090185263,0.1377341909814736],predicted_final_status: 0
1,[1250.0,1.0,1302669381.0,1302669382.0,1300233075.0,1300768581.0,27.0]-->prob=[0.35393964858022414,0.6460603514197759],predicted_final_status: 1
0,[1250.0,1.0,1310090400.0,1310090432.0,1307355515.0,1307388490.0,6.0]-->prob=[0.9156725957387367,0.08432740426126324],predicted_final_status: 0
0,[1250.0,1.0,1310830681.0,1310830685.0,1307990451.0,1308238681.0,0.0]-->prob=[0.9240554703980411,0.07594452960195879],predicted_final_status: 0
0,[1250.0,1.0,1313534846.0,1313534847.0,1307929849.0,1310942846.0,8.0]-->prob=[0.8932189092205174,0.10678109077948258],predicted_

1,[2000.0,1.0,1369590830.0,1369590830.0,1366652924.0,1366998830.0,31.0]-->prob=[0.3444891372770242,0.6555108627229759],predicted_final_status: 1
0,[2000.0,1.0,1369746000.0,1369746038.0,1366170021.0,1369169369.0,24.0]-->prob=[0.37554148484922883,0.6244585151507712],predicted_final_status: 1
0,[2000.0,1.0,1369746786.0,1369746787.0,1367186707.0,1367932386.0,8.0]-->prob=[0.9456815345042495,0.05431846549575057],predicted_final_status: 0
0,[2000.0,1.0,1369841884.0,1369841884.0,1366771084.0,1367249884.0,12.0]-->prob=[0.9522488868025804,0.047751113197419655],predicted_final_status: 0
0,[2000.0,1.0,1369907516.0,1369907516.0,1366060695.0,1367315516.0,0.0]-->prob=[0.9791460092307098,0.020853990769290308],predicted_final_status: 0
1,[2000.0,1.0,1370014113.0,1370014115.0,1363981850.0,1367422113.0,31.0]-->prob=[0.3444891372770242,0.6555108627229759],predicted_final_status: 1
1,[2000.0,1.0,1370019600.0,1370019637.0,1362171568.0,1368029467.0,59.0]-->prob=[0.254989513631485,0.745010486368515],predicted

1,[3000.0,1.0,1355108400.0,1355108433.0,1352225450.0,1352558571.0,56.0]-->prob=[0.33660702877273035,0.6633929712272697],predicted_final_status: 1
0,[3000.0,1.0,1355428406.0,1355428406.0,1337630022.0,1352836406.0,2.0]-->prob=[0.9556801778391317,0.04431982216086828],predicted_final_status: 0
0,[3000.0,1.0,1355541644.0,1355541644.0,1353962455.0,1354332044.0,2.0]-->prob=[0.9556801778391317,0.04431982216086828],predicted_final_status: 0
1,[3000.0,1.0,1355931301.0,1355931301.0,1344285965.0,1353339301.0,38.0]-->prob=[0.40259507762733965,0.5974049223726603],predicted_final_status: 1
0,[3000.0,1.0,1355959458.0,1355959460.0,1353300864.0,1353367458.0,1.0]-->prob=[0.9556801778391317,0.04431982216086828],predicted_final_status: 0
0,[3000.0,1.0,1356235200.0,1356235233.0,1353626702.0,1354055316.0,74.0]-->prob=[0.33660702877273035,0.6633929712272697],predicted_final_status: 1
1,[3000.0,1.0,1356386400.0,1356386433.0,1353004089.0,1353968568.0,3042.0]-->prob=[0.1902076010116748,0.8097923989883252],predic

...

1,[500.0,1.0,1404594294.0,1404594294.0,1403985278.0,1403989494.0,41.0]-->prob=[0.3675192244428339,0.632480775557166],predicted_final_status: 1
1,[500.0,1.0,1404694800.0,1404694811.0,1391991196.0,1401642837.0,167.0]-->prob=[0.16518357923203922,0.8348164207679608],predicted_final_status: 1
0,[500.0,1.0,1404855094.0,1404855094.0,1403554111.0,1403559094.0,7.0]-->prob=[0.7567996090068113,0.24320039099318866],predicted_final_status: 0
1,[500.0,1.0,1405367721.0,1405367721.0,1398273948.0,1400183721.0,17.0]-->prob=[0.4183662556940854,0.5816337443059146],predicted_final_status: 1
0,[500.0,1.0,1405550640.0,1405550640.0,1404341052.0,1404346774.0,1.0]-->prob=[0.9397683349738839,0.06023166502611599],predicted_final_status: 0
0,[500.0,1.0,1405622889.0,1403030989.0,1402771639.0,1403030889.0,0.0]-->prob=[0.9397683349738839,0.06023166502611599],predicted_final_status: 0
1,[500.0,1.0,1405945984.0,1405945985.0,1399894188.0,1403526784.0,35.0]-->prob=[0.4287647878371302,0.5712352121628698],predicted_final_s

0,[1200.0,1.0,1418338189.0,1418338189.0,1415061207.0,1415746189.0,2.0]-->prob=[0.9163378291958457,0.08366217080415421],predicted_final_status: 0
1,[1200.0,1.0,1418439600.0,1418439611.0,1416362343.0,1416581724.0,15.0]-->prob=[0.7855899778618545,0.21441002213814558],predicted_final_status: 0
0,[1200.0,1.0,1419112800.0,1419112811.0,1414600501.0,1414603027.0,39.0]-->prob=[0.4824807697218949,0.5175192302781051],predicted_final_status: 1
0,[1200.0,1.0,1420497697.0,1420497697.0,1415063950.0,1417905697.0,1.0]-->prob=[0.9163378291958457,0.08366217080415421],predicted_final_status: 0
0,[1200.0,1.0,1420732384.0,1420732384.0,1417650026.0,1418140384.0,16.0]-->prob=[0.7855899778618545,0.21441002213814558],predicted_final_status: 0
0,[1200.0,1.0,1423327757.0,1423327757.0,1376182089.0,1420735757.0,3.0]-->prob=[0.9108010900397335,0.08919890996026641],predicted_final_status: 0
0,[1200.0,1.0,1423709929.0,1423709930.0,1422843215.0,1422845929.0,0.0]-->prob=[0.9163378291958457,0.08366217080415421],predicted

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)




0,[30000.0,1.0,1421280000.0,1421280011.0,1416318825.0,1420566397.0,4.0]-->prob=[0.9890591964378663,0.010940803562133658],predicted_final_status: 0
0,[30000.0,1.0,1421368460.0,1421368460.0,1404471113.0,1416184460.0,0.0]-->prob=[0.9706452623983269,0.02935473760167303],predicted_final_status: 0
1,[30000.0,1.0,1421778373.0,1421778373.0,1400691369.0,1417717573.0,1955.0]-->prob=[0.35230974829121664,0.6476902517087834],predicted_final_status: 1
0,[30000.0,1.0,1422044671.0,1422044672.0,1413929669.0,1419452671.0,57.0]-->prob=[0.759536539227555,0.2404634607724451],predicted_final_status: 0
0,[30000.0,1.0,1422054038.0,1422054039.0,1416577869.0,1416870038.0,37.0]-->prob=[0.759536539227555,0.2404634607724451],predicted_final_status: 0
0,[30000.0,1.0,1422135191.0,1420563994.0,1417811762.0,1418247191.0,0.0]-->prob=[0.9935513023848548,0.006448697615145193],predicted_final_status: 0
0,[30000.0,1.0,1422210914.0,1422210916.0,1417468188.0,1418322914.0,7.0]-->prob=[0.9862635233798904,0.013736476620109487]

0,[100000.0,1.0,1424374114.0,1424374116.0,1369353456.0,1421782114.0,0.0]-->prob=[0.9909165709533931,0.009083429046606872],predicted_final_status: 0
0,[100000.0,1.0,1424479962.0,1424479962.0,1421722774.0,1421887962.0,0.0]-->prob=[0.9935513023848548,0.006448697615145193],predicted_final_status: 0
0,[100000.0,1.0,1424881977.0,1424881978.0,1418177610.0,1422289977.0,10.0]-->prob=[0.9734702588115868,0.026529741188413208],predicted_final_status: 0
0,[100000.0,1.0,1424959459.0,1424959460.0,1421935050.0,1422367459.0,2.0]-->prob=[0.9935513023848548,0.006448697615145193],predicted_final_status: 0
0,[100000.0,1.0,1424973637.0,1424973641.0,1420818431.0,1421085637.0,0.0]-->prob=[0.9935513023848548,0.006448697615145193],predicted_final_status: 0
0,[100000.0,1.0,1425147223.0,1425147227.0,1422550042.0,1422555223.0,0.0]-->prob=[0.9935513023848548,0.006448697615145193],predicted_final_status: 0
0,[100000.0,1.0,1425250023.0,1425250027.0,1422567597.0,1422658023.0,1.0]-->prob=[0.9935513023848548,0.006448697

In [20]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
# Select (prediction, true label) and compute test error
evaluator = MulticlassClassificationEvaluator(labelCol="indexedLabel", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(Predictions)
print("Test Error = %g" % (1.0 - accuracy))
print("Test Accuracy = %g" % accuracy)

rfModel = PipelineModel.stages[2]
print(rfModel)  # summary only

# # Transfer final_status(dataframe) into RDD
# ori_result = testData.rdd.map(lambda p: float(p.final_status))
# pre_result = Predictions.rdd.map(lambda p: float(p.final_status))
# # print(ori_result)  #RDD
# # print(pre_result)  #RDD
# labelsAndPredictions = ori_result.zip(pre_result)
# print(labelsAndPredictions.take(10))

# # first metrics
# from pyspark.mllib.evaluation import BinaryClassificationMetrics
# metrics = BinaryClassificationMetrics(labelsAndPredictions)
 
# # Area under precision-recall curve
# print("Area under PR = %s" % metrics.areaUnderPR)
 
# # Area under ROC curve
# print("Area under ROC = %s" % metrics.areaUnderROC)

# # second metrics
# accuracyTest = 1.0 * labelsAndPredictions.filter(lambda xv: xv[0] == xv[1]).count() / testData.count()
# print ("Test Accuracy For Random Forest = "  + str(accuracyTest) + "\n")


Test Error = 0.143028
Test Accuracy = 0.856972
RandomForestClassificationModel (uid=RandomForestClassifier_4565b38a0a5d68ab5e04) with 10 trees
